# Provision and Run with Kubernetes

With NVIDIA FLARE, it is possible to deploy an FL system to an existing Kubernetes (k8s) cluster. In this notebook, we will walk through how to provision and run an FL system in k8s managed by [`microk8s`](https://microk8s.io/).

> **Note**:
>
> To follow along with the example provided in this notebook, you need access to a running k8s system. See the section below on how to install and set one up on Ubuntu.
>
> This notebook provides a reference k8s deployment example. Depending on the Kubernetes cluster in your real-world use case, your may need to modify the deployment chart, or perform additional operations for a successful deployment.

We will cover the following items:
- Prepare for k8s deployment
- Provision and Generate Helm Chart
- Deploy Helm Chart
- Verify Deployment and Check Status

> **Note:**
>
> We will only deploy the servers and overseer to k8s, not the clients, since this is what usually makes sense in a real deployment. Clients can join the FL system in a distributed way, using for instance the shell script in their start up kit.

Let's get started.

# Prepare for k8s Deployment

### 1. Install and Set Up `microk8s`

You should first have k8s installed and set up on your system. In this notebook, we use the [`microk8s`](https://microk8s.io/) on an Ubuntu system. Check out [this tutorial](https://ubuntu.com/tutorials/install-a-local-kubernetes-with-microk8s) on how to install it on an Ubuntu system. Make sure you finish all the post-install steps, including setting up the firewall configuration, adding current user to microk8s group etc.

Once you've installed `microK8s` and performed all the post-install steps, you need to make sure that the following services are enabled:

```
    dns                  # (core) CoreDNS
    ha-cluster           # (core) Configure high availability on the current node
    helm3                # (core) Helm 3 - Kubernetes package manager
    hostpath-storage     # (core) Storage class; allocates storage from host directory
    ingress              # (core) Ingress controller for external access
    registry             # (core) Private image registry exposed on localhost:32000
    storage              # (core) Alias to hostpath-storage add-on, deprecated
```
You can check which services are enabled in `microk8s` by typing the following command in a terminal:
```
microk8s status
```
If some of the services are not enabled, enable them with:
```
microk8s enable [SERVICE NAME]
```
For instance, use `microk8s enable ingress` to enable the ingress service.

### 2. Allow Network Traffic

We have to change the k8s cluster to allow incoming network traffic to enter the cluster, such as those from admin consoles and NVIDIA FLARE clients. After the network traffic enters the cluster, the cluster also needs to know how to route the traffic to the deployed services.

To do that, we need to first edit the configmap of the ingress service. Execute the following command in a terminal:
```bash
microk8s kubectl edit cm nginx-ingress-tcp-microk8s-conf -n ingress
```
This will open a configuration file for the ingress with your default text editor. Then add the following lines to the end of this file:
```yaml
data:
    "8002": default/server1:8002
    "8003": default/server1:8003
    "8102": default/server2:8102
    "8103": default/server2:8103
    "8443": default/overseer:8443
```
Save and close the file.

Next, we need to edit the DaemonSet of ingress to open ports. Execute the following command in a terminal:
```bash
microk8s kubectl edit ds nginx-ingress-microk8s-controller -n ingress
```
Similarly, this will also open a configuration file with your default editor. In the configuration file, you need to look for the section `spec.template.spec.containers.ports`, which should contains some port configurations. Add the following to the existing port configurations:
```yaml
- containerPort: 8443
  hostPort: 8443
  name: overseer
  protocol: TCP
- containerPort: 8002
  hostPort: 8002
  name: server1fl
  protocol: TCP
- containerPort: 8003
  hostPort: 8003
  name: server1adm
  protocol: TCP
- containerPort: 8102
  hostPort: 8102
  name: server2fl
  protocol: TCP
- containerPort: 8103
  hostPort: 8103
  name: server2adm
  protocol: TCP
```
Save and close the file.

Restart `microk8s` so the modifications could take effect:
```bash
microk8s stop
microk8s start
```

### 3. Prepare a Docker Image

We need to provide a Docker image that contains NVIDIA FLARE and dependencies, so that the deployed services can use to spin up containers. Since we've already built one such image `nvflare-pt-docker` during [Provision and Run with Docker](../04.4_provision_and_run_with_docker/provision_and_run_with_dockers.ipynb#Build-a-Docker-Image), let's go ahead and use this image.

For `microk8s` to successfully recognize the image, we need to first re-tag the image as `localhost:32000/nvflare-pt-docker` and push it to the `microk8s` internal local registry. Let's execute the following commands in a terminal:

```
docker tag nvflare-pt-docker localhost:32000/nvflare-pt-docker
docker push localhost:32000/nvflare-pt-docker
```
> **Note**: make sure you have the `registry` service up-and-running in `microk8s`. The registry service will listen to port 32000 on localhost.

It could take a couple of minutes before `microk8s` finishes copying the image to it's local registry.

That's all for preparation, we are ready to proceed with project provisioning.

# Provision and Generate Helm Chart

To be able to deployment an FL system in k8s, the first thing we need to do is to generate a [Helm Chart](https://www.redhat.com/en/topics/devops/what-is-helm). 

In NVIDIA FLARE, we can custom the project configuration file and let the provision process generate a Helm Chart, by adding the following in the builder section of the configuration file:
```yaml
- path: nvflare.lighter.impl.helm_chart.HelmChartBuilder
    args:
    docker_image: localhost:32000/nvflare-pt-docker
```

The `docker_image` is the image that we prepared during the previous section, and will be used for all pods running in the k8s.

An example configuration file that includes the `HelmChartBuilder` is provided in [`code/project.yml`](code/project.yml). This configuration file was set to enable the high-availability mode. Let's provision a project with this configuration file:

In [ ]:
!nvflare provision -p code/project.yml -w /tmp/workspace

Inside of the provision workspace, you can find generate packages for all participants, as well as the Helm Chart (folder `nvflare_hc`):

In [ ]:
!tree /tmp/workspace/example_project/prod_00/ -L 1

Feel free to modify the contant of the Helm Chart to custom it. Once you are done, we can proceed to the deployment.

# Deploy Helm Chart

Run the following commands in a terminal to deploy the Helm Chart to k8s:
```bash
mkdir -p /tmp/nvflare
cd /tmp/workspace/example_project/prod_00/
microk8s helm3 install --set workspace=$(pwd) --set persist=/tmp/nvflare nvflare-helm-chart-demo nvflare_hc/
```

Note that we set `workspace/example_project/prod_00/`, which contains all provisioned packages as the `workspace` inside k8s, and mount `/tmp/nvflare` from the host system as a persistent volume in k8s. The name of the application we are deploying is `nvflare-helm-chart-demo`. Feel free to modify these settings.

Once the deployment is finished, you should see output similar to the following:
```
NAME: nvflare-helm-chart-demo
LAST DEPLOYED: Fri Feb 21 00:59:05 2025
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
```

And you can check that the application is deployed in k8s using the following command:
```bash
microk8s helm3 list -A
```

# Verify Deployment and Check Status

To verify the deployment, use the following command:
```bash
microk8s kubectl get pods
```
This should list the `overseer`, `server1` and `server2` as running pods:
```
NAME                        READY   STATUS    RESTARTS   AGE
overseer-5cfbf5465b-v4fpn   1/1     Running   0          14s
server1-768bc648c5-ddtvn    1/1     Running   0          14s
server2-5d899d4856-dvrf7    1/1     Running   0          14s
```

If any of the pods display error status, you can debug all pods status with:
```bash
microk8s kubectl describe pods
```

You can also debug individual pod using:
```bash
microk8s kubectl logs [POD_NAME]
```
where `[POD_NAME]` is the `NAME` displayed with `microk8s kubectl get pods`.

To functionally check that the servers and overseer is running normally, you can try connecting as admin user to the FL system using FLARE Console. 

Launch the FLARE console with the `fl_admin.sh` script inside the admin startup kit (located under `workspace/example_project/prod_00/admin@nvidia.com`). When prompt to enter the admin's email address, enter `admin@nvidia.com`. Once inside the FLARE Console, use the `check_status server` command to see the status.

Note that you might need to modify the `/etc/hosts` of the system which you are using to run the FLARE Console, by adding `[IP_AADRESS] overseer server1 server2` to the file. The `[IP_AADRESS]` is the IP address of the system running the k8s deployment.


**That's it, we've learned how to provision a project and deploy the servers and overseer in k8s!**

If you wish to do some clean up, use the following commands to delete all deloyments and uninstall the deployed application:
```bash
microk8s kubectl delete --all deployments
microk8s helm3 uninstall nvflare-helm-chart-demo
```

# What's Next

We have finished all the content for this chapter, let's have [a recap on what we've learned](../04.8_recap/recap.ipynb)!